In [1]:
import sys
sys.path.append("/home/luis/shrec_modules/cvae_shrec")
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
from modules.shrec import utils,cvae_parameters, cvae, experiment, experiment_parameters, shrec2019utils, generator_images_rotate, cvae_vgg

Using TensorFlow backend.


## Define the parameters for training
Notebook for training the Conditional Variational Autoencoder with a VGG network [Kalliatakis G.](https://github.com/GKalliatakis/Keras-VGG16-places365) pretrained on the [Places](http://places2.csail.mit.edu/) dataset as the encoding neural network. 
The parameters for training. The number of encoding layers gets overriden by the VGG encoding network parameters. The decoding neural network is an inverse VGG-like network as in the train_simple_vae notebook. Other parameters include:

- **epochs**: Number of epochs for training
- **batch_size**: Number of images per batch of training
- **type_layer**: Type of layers used in the encoding and decoding networ. Either "convolutional" or "dense"
- **num_encoding_layers**: Number of the max pooling layers in the encoding network
- **num_decoding_layers**: Number of the upsampling layers in the decoding network
- **var_x**: Variance of the normal distribution for p(x|z,y)
- **r_loss**: Type of loss chosen for reconstruction (either binary cross entropy "binary" or mean squared error "mse")
- **latent_dim**: Dimensions of the latent space of the conditional variational autoencoder
- **intermediate_dim** Defines the number of filters in the case of the convolutions or the number of neurons in a convolutional or dense layer respectively.
- **learning_rate**: Learning rate of the optimizer used
- **image_shape**: Shape of the images used for training


In [2]:
epochs = 500
batch_size = 32
type_layer = "convolutional"
num_encoding_layers = 5
num_decoding_layers = 5
intermediate_dim  =  64
num_classes = 10
latent_dim = 100
learning_rate = 0.001
image_shape = (256,256,3)
var_x = 1.0

### Define data generator
The training of the network is done via a generator. The generator is created from a list of the paths to the corresponding images. 

In [3]:
main_directory = os.path.dirname(os.getcwd())
dataset_directory = os.path.join(main_directory, "dataset")
path_class_files = os.path.join(dataset_directory, "class_files")
# Read the file with the names of the models and images that will be used for training
cla_file_path_renders = os.path.join(path_class_files, "SceneIBR2018_Model_Training.cla")
cla_file_path_photographs = os.path.join(path_class_files, "SceneIBR2018_Image_Training.cla")

# Paths to the corresponding images folder
renders_path = os.path.join(dataset_directory,"renders_256")
photos_path = os.path.join(dataset_directory,"images_256")

# Renders
categories, model_lists = shrec2019utils.obtain_models_classes(cla_file_path_renders)
categories_array, models_array = shrec2019utils.categories_model_list_to_array(categories, model_lists)
# Photographs
categories_photographs, model_lists_photographs = shrec2019utils.obtain_models_classes2018(cla_file_path_photographs)
categories_array_photographs, models_array_photographs = shrec2019utils.categories_model_list_to_array(categories_photographs, model_lists_photographs)

print("Categories of renders {} amount {}".format(np.unique(categories), len(np.unique(categories))))
print("Categories of images {} amount {}".format(np.unique(categories_photographs), len(np.unique(categories_photographs))))
print("Amount of renders {}".format(len(models_array)))
print("Amount of images {}".format(len(models_array_photographs)))

Initial index barn 0 70
Initial index beach 0 70
Initial index bedroom 0 70
Initial index castle 0 70
Initial index classroom 0 70
Initial index desert 0 70
Initial index kitchen 0 70
Initial index library 0 70
Initial index mountain 0 70
Initial index river 0 70
Initial index barn 0 700
Initial index beach 0 700
Initial index bedroom 0 700
Initial index castle 0 700
Initial index classroom 0 700
Initial index desert 0 700
Initial index kitchen 0 700
Initial index library 0 700
Initial index mountain 0 700
Initial index river 0 700
Categories of renders ['barn' 'beach' 'bedroom' 'castle' 'classroom' 'desert' 'kitchen'
 'library' 'mountain' 'river'] amount 10
Categories of images ['barn' 'beach' 'bedroom' 'castle' 'classroom' 'desert' 'kitchen'
 'library' 'mountain' 'river'] amount 10
Amount of renders 700
Amount of images 7000


### Join identifiers for rendered images and photographs

Create a list comprising both, the identifiers for the renders and the images used for training. 

In [4]:
# Join the paths for the training renders and photographs
images_paths = []
joint_categories = []
type_data = []
for num_model, model in enumerate(models_array):
    for image_path in glob.glob(os.path.join(renders_path,  str(model)+'_*.png')):
        images_paths.append(image_path)
        joint_categories.append(categories_array[num_model])
        type_data.append("render")
number_of_renders = len(images_paths)
print("Amount of renders ",number_of_renders)   
for num_model, model in enumerate(models_array_photographs):
    for image_path in glob.glob(os.path.join(photos_path,  str(model)+'.png')):
        images_paths.append(image_path)
        joint_categories.append(categories_array_photographs[num_model])
        type_data.append("photograph")
number_of_photographs = len(images_paths)-number_of_renders
print("Amount of photographs ",number_of_photographs)   



Amount of renders  9100
Amount of photographs  7000


In [5]:
# Join the categories for each of the paths
joint_categories = np.array(joint_categories)
joint_categories = shrec2019utils.categories_array_to_numerical(joint_categories)
# Transform the categories to one hot encodings
joint_categories = utils.one_hotify(joint_categories)
print("Joint categories length {}".format(len(joint_categories)))
print("Joint paths length {}".format(len(images_paths)))

# Create the generator

dim = (image_shape[0], image_shape[1]) # resolution of the images
n_channels = image_shape[2] # number of color channels
generator = generator_images_rotate.ShrecGenerator(images_paths, joint_categories,type_data, dim= dim, n_channels = n_channels, batch_size=batch_size, n_classes = num_classes, flatten=False, resize=True)




Joint categories length 16100
Joint paths length 16100


### Training of Conditional Variational Autoencoder
In this section an instance of the conditional variational autoencoder class is defined with respect to the parameters previously defined. The class containes the encoding and decoding neural network of the conditional variational autoencoder. 


In [6]:
# DEFINE THE NETWORK OF THE CONDITIONAL VARIATIONAL AUTOENCODER
cvae_parameters_dict = {"image_shape": image_shape,# shape of the input images
                       "num_classes": num_classes, # number of classes for the scenes
                       "type_layers":type_layer,# either "convolutional neural network" or "dense neural network"
                       "num_encoding_layers": num_encoding_layers, # number of the max pooling layers in the encoding
                       "num_decoding_layers": num_decoding_layers, # number of the upsampling layers in the decoding
                       "var_x":var_x, # variance of the normal distribution for p(x|z,y)
                       "r_loss":"mse", # type of loss chosen (either binary cross entropy "binary" or mean squared error "mse")
                       "latent_dim":latent_dim, # dimensions of the latent space of the conditional variational autoencoder
                       "intermediate_dim":intermediate_dim, # defines the number of filters in the convolutions
                       "learning_rate":learning_rate # learning rate of the optimizer
                       }
# Define the parameters for the CVAE
parameters = cvae_parameters.CVAEParams(**cvae_parameters_dict)
# Define the CVAE
cond_vae = cvae_vgg.CVAE_VGG(parameters)

W1028 16:36:42.434905 139956805080896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Reconstruction loss is mean squared error
Reconstruction loss is mean squared error


### Start training experiment
Define an experiment for training. The experiment takes the instance of the conditional variational autoencoder, the experiment parameters and a folder for saving the tensorboard log files and the trained model weights.



In [ ]:
# Define the parameters for training
experiment_params_dict = {"epochs": epochs,
                         "batch_size": batch_size}
exp_parameters  = experiment_parameters.ExperimentParams(**experiment_params_dict)

# Define an experiment
saving_model_folder = os.path.join(main_directory, "trained_models","vgg_submission")


exp = experiment.Experiment(cond_vae, exp_parameters, None, saving_model_folder)

exp.run_image_generator(generator) # Training of the CVAE with the generator as a source of data

W1028 16:36:49.757392 139956805080896 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1028 16:36:52.139703 139956805080896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W1028 16:36:52.147371 139956805080896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W1028 16:36:52.148162 139956805080896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks/tensorboard_v

Epoch 1/500
 - 422s - loss: 1500.2850 - r_loss: 1487.0688 - kl_loss: 10.6430 - mean_squared_error: 0.0151 - classification_loss: 2.3392


W1028 16:43:55.099596 139956805080896 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks/tensorboard_v1.py:343: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



Epoch 2/500
 - 427s - loss: 1459.2462 - r_loss: 1456.6420 - kl_loss: 0.0658 - mean_squared_error: 0.0148 - classification_loss: 2.3024
Epoch 3/500
